## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [1]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

mkdir: cannot create directory ‘pw-data’: File exists
File ‘./pw-data/201701scripts_sample.json.gz’ already there; not retrieving.

File ‘./pw-data/practices.json.gz’ already there; not retrieving.



## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [2]:
import gzip
import simplejson as json

In [3]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

In [5]:
scripts[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

In [6]:
practices[:2]

[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

## 1. Summary_Statistics

(`scripts`) data contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`).

Here sum, mean, standard deviation, and quartile statistics for each of these quantities is calculated. Results are formated for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. At the end a `tuple` with these lists for each quantity is calculated.

In [10]:
# ***************************************  SCRIPTS DATA *************************************************************************
# CACLCULATION OF sum, mean, standard deviation, 1st quartile, median, 3rd quartile OF SCRIPTS DATA


import math

def describe(key):
    total = 0
    avg = 0
    s = 0
    q25 = 0
    med = 0
    q75 = 0
    
    list1 = [script[key] for script in scripts]
    list1.sort()
    length2 = len(list1)
#     TOTAL This is the Total  ****************************************** 
    total = sum(list1) 
    
#     AVERAGE This is the Average  ******************************************
    avg = (total) / len(scripts)
    
#     STANDARD DEVIATION CALCULATION
    sum1 = []
    for script in scripts:
        sum1.append((script[key] - avg)**2)
    sum_of_list = sum(sum1)
    take_sqrt = (sum_of_list/ len(scripts)-1)
    s = math.sqrt(take_sqrt)  
     
#     Quartile no 1 q25        ************************************************************************

    pos25 = math.floor(len(list1)*0.25)
    q25 = list1[pos25-1]


#     Median This is the Median      ************************************************************************
    length = len(list1)
    if(length % 2 == 0):
        divide = int(length / 2)
        value1 = list1[divide-1]
        value2 = list1[divide]
        median = ((value1 + value2)/2)
    else:
        divide = length / 2
        value1 = math.floor(divide)
        median = list1[value1]
    
#     Quartile no 2 q75 This is quartile no 2 ************************************************************************                 
    pos75 = math.floor(len(list1)* 0.75)
    q75 = list1[pos75]
    
#   At the end i make another list (list2) and then add all the values to it as mentioned in the question and return this list2
    list2 = [total, avg, s, q25, median, q75]
    return list2


In [11]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

In [12]:
print(summary)

[('items', [4410054, 11.522744731217633, 33.097062705243616, 1, 3.0, 8]), ('quantity', [316356836, 826.5883059943667, 3872.1808854834294, 30, 120.0, 466]), ('nic', [29048309.790000085, 75.89844899484248, 197.57029589915396, 7.7, 22.62, 65.94]), ('act_cost', [27053937.60000044, 70.68748295125087, 183.26459067761738, 7.25, 21.24, 61.53])]


## 2. Most_Common_Item

we are interested in particular groups in scripts -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Here we calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data? ori.e. the max_item, or most_recurring?

To calculate this, we first split our data set into groups corresponding with the different values of `'bnf_name'`. Then sum the number of items dispensed within in each group where the largest sum can be found.

`'bnf_name'` is used to construct our groups. There are *5619* unique values for `'bnf_name'`.

In [13]:
scripts[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

In [42]:
# 1st simple implementation without using a function
list1 = []
dict1 = {}
for script in scripts:
    if(script['bnf_name'] not in list1):
        list1.append(script['bnf_name'])

In [19]:
bnf_names = set([script['bnf_name'] for script in scripts])

In [20]:
assert(len(bnf_names) == 5619)

In [21]:
type(bnf_names)

set

In [41]:
len(bnf_names)

5619

In [24]:
type(bnf_names)

set

In [26]:
list(bnf_names)[:4]

['Difflam_Sore Throat Rinse 0.15%',
 'Telfast 120_Tab 120mg',
 'Nitrofurantoin_Tab 50mg',
 'Apixaban_Tab 5mg']

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [28]:
groups = {name: [] for name in bnf_names}

In [29]:
type(groups)

dict

In [30]:
list(groups.items())[:2]

[('Difflam_Sore Throat Rinse 0.15%', []), ('Telfast 120_Tab 120mg', [])]

In [31]:
for script in scripts:
    groups[script['bnf_name']].append(script)

In [32]:
max_item = [max(
    (sum(member['items'] for member in group), name)
    for name, group in groups.items()
)[::-1]]

In [34]:
max_item    # Most Common Item

[('Omeprazole_Cap E/C 20mg', 113826)]

In [44]:
# 2nd implementation to find most common item using a function most_common_item()

def most_common_item():
    groups = {name: [] for name in bnf_names} 
    for script in scripts:
         groups[script['bnf_name']].append(script)
    max1=0
    for i in groups:
        summ=0
        for j in range(len(groups[i])):
            summ+=(groups[i][j]['items'])
        if summ>=max1:
            max1=summ
            name1=i
    return [(tuple((name1, max1)))]

In [45]:
max_item = most_common_item()

In [67]:
print(max_item)

[('Omeprazole_Cap E/C 20mg', 113826)]


## 3. Postal_Totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Here the objective is to find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices'`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [68]:
scripts[0]   # For Reference

{'bnf_code': '0101010G0AAABAB',
 'items': 2,
 'practice': 'N81013',
 'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
 'nic': 5.98,
 'act_cost': 5.56,
 'quantity': 1000,
 'post_code': 'SK11 6JL'}

In [69]:
practices[0]  # For Reference

{'code': 'A81001',
 'name': 'THE DENSHAM SURGERY',
 'addr_1': 'THE HEALTH CENTRE',
 'addr_2': 'LAWSON STREET',
 'borough': 'STOCKTON ON TEES',
 'village': 'CLEVELAND',
 'post_code': 'TS18 1HU'}

In [70]:
practice_postal = {}
for practice in practices:
    code = practice['code']
    post_code = practice['post_code']
    
    if code not in practice_postal:
        practice_postal[code] = post_code
        
    if code in practice_postal:
        first_post_code = min(post_code, practice_postal[code])
        practice_postal[code] = first_post_code

In [71]:
items_list = [script['items'] for script in scripts]

In [72]:
items_list[:10]

[2, 1, 12, 3, 6, 15, 5, 10, 2, 1]

**NOTE:** This is an aggregation of the practice data grouped by practice codes. Here we write an alternative implementation of the above cell using a `group_by_field` function.

In [73]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we join `practice_postal` to `scripts`.

In [74]:
joined = scripts[:]
for script in joined:
    script['post_code'] = practice_postal[script['practice']]

In [75]:
joined[0]

{'bnf_code': '0101010G0AAABAB',
 'items': 2,
 'practice': 'N81013',
 'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
 'nic': 5.98,
 'act_cost': 5.56,
 'quantity': 1000,
 'post_code': 'SK11 6JL'}

Finally we group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous `most_common_item` exercise.

In [76]:
def group_by_field(data, fields):
    names = {tuple(dict_[field] for field in fields)
            for dict_ in data}
    groups = {name: [] for name in names}
    for dict_ in data:
        name = tuple(dict_[field] for field in fields)
        groups[name].append(dict_)
    return groups

In [77]:
items_by_post = []

for name, group in list(group_by_field(joined, ('post_code',)).items()):
    item_sum = sum(dict_['items'] for dict_ in group)
    name = name[0]
    my_tuple = (name, item_sum)
    items_by_post.append(my_tuple)

In [78]:
items_by_post.sort()

In [79]:
postal_totals = items_by_post[:100]

In [80]:
print(postal_totals)

[('B11 4BW', 20673), ('B18 7AL', 19001), ('B21 9RY', 29103), ('B23 6DJ', 24859), ('B70 7AW', 36531), ('BB11 2DL', 34356), ('BB2 1AX', 28254), ('BB3 1PY', 54514), ('BB4 5SL', 29388), ('BB7 2JG', 44585), ('BB8 0JZ', 54380), ('BB9 7SR', 38224), ('BD3 8QH', 21010), ('BH18 8EE', 39413), ('BH23 3AF', 32545), ('BL1 8TU', 26132), ('BL3 5HP', 27147), ('BL9 0NJ', 32062), ('BL9 0SN', 35275), ('CB9 8HF', 51337), ('CH1 4DS', 34915), ('CH65 6TG', 25090), ('CT11 8AD', 44358), ('CV1 4FS', 37210), ('CW1 3AW', 64104), ('CW5 5NX', 38797), ('CW7 1AT', 43164), ('DA1 2HA', 26075), ('DA11 8BZ', 24090), ('DN22 7XF', 43091), ('DN34 4GB', 48013), ('FY2 0JG', 69118), ('FY4 1TJ', 62886), ('FY5 2TZ', 44258), ('FY7 8GU', 34473), ('GL1 3PX', 38120), ('GL50 4DP', 74822), ('GU9 9QS', 32131), ('HA0 4UZ', 22755), ('HA3 7LT', 32113), ('HG1 5AR', 32684), ('HU7 4DW', 49107), ('KT14 6DH', 26758), ('KT6 6EZ', 38975), ('L31 0DJ', 32065), ('L36 7XY', 22965), ('L5 0QW', 24676), ('L7 6HD', 42569), ('LA1 1PN', 47335), ('LE10 1DS'

## 4. Items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We will use the `joined` variable we created before in previous question, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

**NOTE:** The output should be like below, reference of the expected output is given below i.e. FROM, TO

```python 

FROM

{
('CH1 4DS', 'GlucoRx FinePoint Ultra Needles Pen Inj')  : [{'bnf_code': '21010900697',
                                                               'items': 1,
                                                               'practice': 'N81080',
                                                               'bnf_name': 'GlucoRx FinePoint Ultra Needles Pen Inj',
                                                               'nic': 11.9,
                                                               'act_cost': 11.03,
                                                               'quantity': 200,
                                                               'post_code': 'CH1 4DS'}, {'bnf_code': '21010900697',
                                                               'items': 1,
                                                               'practice': 'N81102',
                                                               'bnf_name': 'GlucoRx FinePoint Ultra Needles Pen Inj',
                                                               'nic': 11.9,
                                                               'act_cost': 11.03,
                                                               'quantity': 200,
                                                               'post_code': 'CH1 4DS'}]
}

TO

{
('CH1 4DS', 'GlucoRx FinePoint Ultra Needles Pen Inj')  : 2
}

```

In [81]:
total_items_by_bnf_post = {}

for key, group in list(group_by_field(joined, ('post_code', 'bnf_name')).items()):
    items_total = sum(d['items'] for d in group)
    total_items_by_bnf_post[key] = items_total
    

In [82]:
type(total_items_by_bnf_post)

dict

In [83]:
total_items_by_bnf_post[('CH1 4DS', 'GlucoRx FinePoint Ultra Needles Pen Inj')]

5

In [84]:
len(total_items_by_bnf_post)

141196

In [85]:
# total_items_by_bnf_post = ...
assert len(total_items_by_bnf_post) == 141196

Let's use `total_by_item_post` to find the maximum item total for each postal code. To do this, we will want to regroup `total_by_item_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_by_item_post` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in `total_by_item_post` after grouping it by `'post_code'`.

In [86]:
total_items = []
for (post_code, bnf_name), total in list(total_items_by_bnf_post.items()):
    new_dict = {'post_code': post_code,
               'bnf_name': bnf_name,
                'total': total}
    total_items.append(new_dict)

In [87]:
total_items[:2]

[{'post_code': 'B23 6DJ', 'bnf_name': 'Hydrocort_Crm 0.5%', 'total': 10},
 {'post_code': 'NN16 8DN', 'bnf_name': 'Atenolol_Tab 50mg', 'total': 243}]

In [88]:
total_items_by_post = group_by_field(total_items, ('post_code',))

In [89]:
len(total_items_by_post)

118

In [90]:
assert len(total_items_by_post) == 118

```python
 total_items_by_post sample output

{('BB8 0JZ',): [{'post_code': 'BB8 0JZ',
   'bnf_name': 'Pseudoephed HCl_Tab 60mg',
   'total': 3},
  {'post_code': 'BB8 0JZ',
   'bnf_name': 'Co-Danthramer_Susp 25mg/200mg/5ml S/F',
   'total': 3},
  {'post_code': 'BB8 0JZ', 'bnf_name': 'E45_Emollient Shower Crm', 'total': 5},
  {'post_code': 'BB8 0JZ',
   'bnf_name': 'Salbutamol_Inha 100mcg (200 D) CFF',
   'total': 1131},
  {'post_code': 'BB8 0JZ', 'bnf_name': 'Salazopyrin-En_Tab 500mg', 'total': 5},
  {'post_code': 'BB8 0JZ', 'bnf_name': 'Methyldopa_Tab 500mg', 'total': 3},
  {'post_code': 'BB8 0JZ',
   'bnf_name': 'Fluconazole_Oral Susp 50mg/5ml',
   'total': 2},
  {'post_code': 'BB8 0JZ', 'bnf_name': 'Spironol_Tab 100mg', 'total': 14},
  {'post_code': 'BB8 0JZ',
   'bnf_name': 'Alprostadil_Inj 20mcg Pfs Cart',
   'total': 3},



````

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [153]:
max_item_by_post = max(total_items_by_post)

In [154]:
max_item_by_post

('YO16 4LZ',)

In [155]:
max_item_by_post[:3]

('YO16 4LZ',)

In [156]:
from operator import itemgetter

In [157]:
get_total = itemgetter('total')

In [158]:
max_item_by_post = []
groups = list(total_items_by_post.values())
for group in groups:
    max_total = sorted(group, key=itemgetter('total'), reverse=True)[0]
    max_item_by_post.append(max_total)


In [159]:
max_item_by_post = [sorted(group, key=itemgetter('total'), reverse=True)[0] for group in list(total_items_by_post.values())]

In [160]:
test_list = total_items_by_post[('YO16 4LZ',)]

In [161]:
get_total(test_list[1])

58

In [162]:
test_list[:5]

[{'post_code': 'YO16 4LZ', 'bnf_name': 'Quetiapine_Tab 50mg M/R', 'total': 2},
 {'post_code': 'YO16 4LZ',
  'bnf_name': 'GlucoRx FinePoint Ultra Needles Pen Inj',
  'total': 58},
 {'post_code': 'YO16 4LZ',
  'bnf_name': 'Beclomet Diprop_Nsl Spy 50mcg (200 D)',
  'total': 161},
 {'post_code': 'YO16 4LZ',
  'bnf_name': 'Movicol Plain_Paed Pdr Sach 6.9g',
  'total': 3},
 {'post_code': 'YO16 4LZ',
  'bnf_name': 'Mebendazole_Oral Susp 100mg/5ml',
  'total': 2}]

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [163]:
type(items_by_post)

list

In [164]:
max_item_by_post[0]

{'post_code': 'BB8 0JZ', 'bnf_name': 'Atorvastatin_Tab 20mg', 'total': 1227}

In [165]:
list(dict(items_by_post))[:3]

['B11 4BW', 'B18 7AL', 'B21 9RY']

In [166]:
items_by_region = []

for item in max_item_by_post:
    numerator = item['total']
    denominator = dict(items_by_post)[item['post_code']]
    proportion = numerator / denominator 
    result = (item['post_code'], item['bnf_name'], proportion)
    items_by_region.append(result)

In [167]:
items_by_region[0]

('BB8 0JZ', 'Atorvastatin_Tab 20mg', 0.022563442442074293)

In [168]:
# only first 100 items_by_region required to display
items_by_region = sorted(items_by_region)[:100]

To check, the output should be like this...


items_by_region = [('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.0341508247)] * 100

In [169]:
items_by_region[:10]

[('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.03415082474725487),
 ('B18 7AL', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.02926161780958897),
 ('B21 9RY', 'Metformin HCl_Tab 500mg', 0.03549462254750369),
 ('B23 6DJ', 'Lansoprazole_Cap 30mg (E/C Gran)', 0.024095900880968663),
 ('B70 7AW', 'Paracet_Tab 500mg', 0.0266896608360023),
 ('BB11 2DL', 'Omeprazole_Cap E/C 20mg', 0.02884503434625684),
 ('BB2 1AX', 'Omeprazole_Cap E/C 20mg', 0.03645501521908402),
 ('BB3 1PY', 'Omeprazole_Cap E/C 20mg', 0.03428477088454342),
 ('BB4 5SL', 'Omeprazole_Cap E/C 20mg', 0.040696883081529876),
 ('BB7 2JG', 'Omeprazole_Cap E/C 20mg', 0.029471795446899183)]